In [1]:
import pandas
import gzip
from collections import defaultdict
import numpy as np


In [2]:
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [3]:
#baselines.py

training_Ratings = []
training_userRatings = defaultdict(list)


validation_userRatings = defaultdict(list)

num_lines = 0

for l in readGz("train.json.gz"):
    user,business = l['reviewerID'],l['itemID']

    if num_lines < 100000:
        training_Ratings.append(l['rating'])
        training_userRatings[user].append(l['rating'])
        num_lines +=1
    else:
        validation_userRatings[user].append(l['rating'])
        
globalAverage = sum(training_Ratings) / len(training_Ratings)
userAverage = {}
for u in training_userRatings:
    userAverage[u] = sum(training_userRatings[u]) / len(training_userRatings[u])

#find rating mse for training and validation data
training_mse = 0
validation_mse = 0
num_lines2 = 0
for l in readGz("train.json.gz"):
    user,business = l['reviewerID'],l['itemID']
    if user in userAverage:
        prediction = userAverage[user]
    else:
        prediction = globalAverage
    if num_lines2 < 100000:
        training_mse += (l['rating'] - prediction)**2
    if num_lines2 >= 100000:
        validation_mse += (l['rating'] - prediction)**2
    num_lines2 +=1

print("Training Mean Squared Error: " + str(training_mse/100000))
print("Validation Mean Squared Error: " + str(validation_mse/100000))

Training Mean Squared Error: 0.6527804770760693
Validation Mean Squared Error: 1.4976080782955097


In [4]:
#Create training and validation set
training_data = {}
validation_data = {}
 
all_users = list()
all_items= list()
    
split=0
#create training and validation set
for l in readGz("train.json.gz"):
    user,item,rating = l['reviewerID'],l['itemID'],l['rating']
    all_users.append(user)
    all_items.append(item)
    
    if split < 100000:
        training_data[(user,item)] = rating
    else:
        validation_data[(user,item)] = rating
    split += 1    
    

In [ ]:
#baseline gets 1.31393 on test data

In [42]:
#latent factor model
import random
#perform grad descent
n=5
P= np.random.rand(len(all_users),n)
Q= np.random.rand(n, len(all_items))
alpha = globalAverage
beta_u = np.random.rand(len(all_users),1)
beta_i = np.random.rand(len(all_items), 1)
index=0
lmbda = .05 #lambda hyperparam
gamma= .01 #gamma hyperparam
iterations = 10000
#try batch grad descent,or  try increasing learning rate and adding regularization terms

print("begin gradient descent")
for i in range(iterations):
    if i %1000 ==0:
        print("iteration number " + str(i) )
        gamma = gamma*.1
   # print("iteration number " + str(i))

    pair = random.choice(list(training_data.keys()))
    r_ui = training_data[pair]
    (userId,itemId) = pair    
    uIndex = all_users.index(userId)    
    iIndex = all_items.index(itemId)
    summation = 0
    q_val = np.zeros((n, len(all_items))) #innefiicient to make 2 different 2 matricies each loop, more effiecent to 
    p_val = np.zeros((len(all_users),n))
    for j in range(n):
        summation += np.dot(P[uIndex][j],Q[j][iIndex])
        q_val += Q[j][iIndex]
        p_val += P[uIndex][j]
    grad_a = 2*(r_ui -(alpha + beta_u[uIndex] +beta_i[iIndex] + summation))*-1 
    grad_bu = 2*(r_ui -(alpha + beta_u[uIndex]+beta_i[iIndex] + summation))*-1
    grad_bi = 2*(r_ui -(alpha + beta_u[uIndex]+beta_i[iIndex] + summation))*-1
    grad_p = 2*(r_ui -(alpha + beta_u[uIndex]+beta_i[iIndex] + summation))*-q_val.T + lmbda*2*p_val
    grad_q = 2*(r_ui -(alpha + beta_u[uIndex]+beta_i[iIndex] + summation))*-p_val.T + lmbda*2*q_val

    
    alpha = alpha - gamma*grad_a
    beta_u[uIndex] = beta_u[uIndex] - gamma*grad_bu #change to only update relevent element of beta vector
    beta_i[iIndex] = beta_i[iIndex] - gamma*grad_bi
    P = P - gamma*grad_p
    Q = Q - gamma*grad_q
    if i %1000 ==0:
        real_rating = training_data[pair]
        pred_rating = alpha[0] + beta_u[uIndex][0] + beta_i[iIndex][0] + np.dot(P[uIndex],(Q.T)[iIndex]) + lmbda*(np.dot(P[userindex],P[userindex])+np.dot((Q.T)[itemindex],(Q.T)[itemindex])) 
        mse = (real_rating - pred_rating)**2
        print("Current MSE: " + str(mse))

        


begin gradient descent
iteration number 0
Current MSE: 12.4667939721232
iteration number 1000
Current MSE: 0.08309732063210085
iteration number 2000
Current MSE: 2.6389477891762247
iteration number 3000
Current MSE: 0.26272260115903334
iteration number 4000
Current MSE: 0.005670757470407415
iteration number 5000
Current MSE: 0.7285970601789586
iteration number 6000
Current MSE: 0.12207133629252477
iteration number 7000
Current MSE: 2.454080578037954
iteration number 8000
Current MSE: 0.24522323442906374
iteration number 9000
Current MSE: 1.522946728517134


In [44]:

training_mse = 0
print("begin calculating training mean squared error")
for pair in training_data.keys():
    real_rating = training_data[pair]
    (userId,itemId) = pair
    userindex = all_users.index(userId)
    itemindex = all_items.index(itemId)
#     print(beta_u.shape)
#     print(P[userindex].shape)
#     print((Q.T)[itemindex].shape)
#     print(np.dot(P[userindex][:],Q[:][itemindex]).shape)
    pred_rating = alpha[0] + beta_u[userindex][0] +beta_i[itemindex][0] + np.dot(P[userindex],(Q.T)[itemindex]) + lmbda*(np.dot(P[userindex],P[userindex])+np.dot((Q.T)[itemindex],(Q.T)[itemindex])) 
    #continuously print mse to ensure it is decreasing
    training_mse += (real_rating- pred_rating)**2
print("Training Data mse: "+ str(training_mse/len(training_data.keys())))

print("begin calculating validation mean squared error")

validation_mse = 0
for pair in validation_data.keys():
    real_rating = validation_data[pair]
    (userId,itemId) = pair
    userindex = all_users.index(userId)
    itemindex = all_items.index(itemId)
    pred_rating = alpha[0] + beta_u[userindex][0]+ beta_i[itemindex][0] + np.dot(P[userindex],(Q.T)[itemindex]) + lmbda*(np.dot(P[userindex],P[userindex])+np.dot((Q.T)[itemindex],(Q.T)[itemindex])) 
    #continuously print mse to ensure it is decreasing
    validation_mse += (real_rating - pred_rating)**2
print("Validation mse: "+ str(validation_mse/len(validation_data.keys())))


begin calculating training mean squared error
Training Data mse: 1.4589487400993177
begin calculating validation mean squared error
Validation mse: 1.4404220129871395


Baseline
Training Mean Squared Error: 0.6527804770760693
Validation Mean Squared Error: 1.4976080782955097

My Results
lambda: .01, gamma .001, iters: 10000, 5 factors
Training Data mse: 1.3465521145092358
begin calculating validation mean squared error
Validation mse: 1.3399293960615437

lambda: .01 gamma .005 iters 10000, 5 factors
Training Data mse: 1.3465207720142207
begin calculating validation mean squared error
Validation mse: 1.3398456298426356


lambda: .01 gamma .01 iters 10000, 5 factors
Training Data mse: 1.386371888971709
begin calculating validation mean squared error
Validation mse: 1.3775317550803412


lambda: .01 gamma .005 iters 10000, 10 factors
Training Data mse: 1.422641247989086
begin calculating validation mean squared error
Validation mse: 1.4126002616984492

lambda: .01 gamma .005 iters 10000, 3 factors
Training Data mse: 1.3488242622374744
begin calculating validation mean squared error
Validation mse: 1.3446991268837358

same as above with 6 latent factors
Training Data mse: 1.3602101491616267
begin calculating validation mean squared error
Validation mse: 1.3520673845715372


lambda: .05 gamma .005 iters 10000, 5 factors
Training Data mse: 1.3626210835902632
begin calculating validation mean squared error
Validation mse: 1.3529848505500985


begin calculating training mean squared error
Training Data mse: 1.4172102100973825
begin calculating validation mean squared error
Validation mse: 1.4152499820427742